In [1]:
#import Pkg
#Pkg.add("JuMP")
#Pkg.add("GLPK")
#Pkg.add("Gruobi")
#Pkg.add("DataFrames")
#Pkg.add("CSV")

In [2]:
using JuMP, GLPK
using DataFrames
using CSV

In [3]:
#load cost data
con_generation = CSV.File("../data/conventional_generators.csv") |> DataFrame

#load wind_technicaldata
wind_generation = CSV.File("../data/wind_farms.csv") |> DataFrame

# load wind profile
wind_profile = CSV.File("../data/wind_powerprofile_200.csv") |> DataFrame

# batteries
battery = CSV.File("../data/Battery.csv") |> DataFrame

# transmission_lines
transmission_lines = CSV.File("../data/transmission_lines.csv") |> DataFrame


Row,Transmission lines: From node,To node,Susceptance [per-unit],Capacity [MW]
,Int64,Int64,Float64,Int64
1,1,2,0.0146,175
2,1,3,0.2253,175
3,1,5,0.0907,350
4,2,4,0.1356,175
5,2,6,0.205,175
6,3,9,0.1271,175
7,3,24,0.084,400
8,4,9,0.111,175
9,5,10,0.094,350


In [4]:
#demand bids
name = "demand_hour_0.csv"
demand_bids = CSV.File("../data/demand_bids_hour/" * name) |> DataFrame
 
# number of convential generators
G = size(con_generation, 1)

# number of demand
D = size(demand_bids, 1)

# number of wind generators
W = size(wind_generation, 1)

# number of batteries
B = size(battery, 1)

# number of transmission lines
L = size(transmission_lines, 1)

# number of nodes
N = 24


24

In [5]:
# Initialize the DataFrame directly without dynamic column names
result_df = DataFrame(hour = Int[])

# For x variables, manually add each column. This is a one-time setup.
for i in 1:G
    result_df[!, Symbol("x_con$i")] = Float64[]
end

# For w variables, manually add each column. This is a one-time setup.
for i in 1:W
    result_df[!, Symbol("x_wind$i")] = Float64[]
end

# For y variables, manually add each column. This is a one-time setup.
for i in 1:D
    result_df[!, Symbol("y$i")] = Float64[]
end

# For b variables, manually add each column. This is a one-time setup.
for i in 1:B
    result_df[!, Symbol("x_bat$i")] = Float64[]
end

# For l variables, manually add each column. This is a one-time setup.
for i in 1:N
    result_df[!, Symbol("x_angle$i")] = Float64[]
end



equilibrium_df = DataFrame(hour = Int[])

# for each node a market price column
for i in 1:N
    equilibrium_df[!, Symbol("market_price_node$i")] = Float64[]
end



In [6]:
println((result_df))

0×63 DataFrame
 Row │ hour   x_con1   x_con2   x_con3   x_con4   x_con5   x_con6   x_con7   x_con8   x_con9   x_con10  x_con11  x_con12  x_wind1  x_wind2  x_wind3  x_wind4  y1       y2       y3       y4       y5       y6       y7       y8       y9       y10      y11      y12      y13      y14      y15      y16      y17      x_bat1   x_bat2   x_bat3   x_bat4   x_bat5   x_angle1  x_angle2  x_angle3  x_angle4  x_angle5  x_angle6  x_angle7  x_angle8  x_angle9  x_angle10  x_angle11  x_angle12  x_angle13  x_angle14  x_angle15  x_angle16  x_angle17  x_angle18  x_angle19  x_angle20  x_angle21  x_angle22  x_angle23  x_angle24 
     │ Int64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64   Float64 

In [7]:
# Load demand bids data
demand_bids_all = [CSV.File("../data/demand_bids_hour/demand_hour_$(i-1).csv") |> DataFrame for i in 1:24]

# Create a new model with GLPK solver
model = Model(GLPK.Optimizer)

# Define the decision variables for every generator and hour
@variable(model, x_con[1:G, 1:24] >= 0)  # Power output variable for conventional generators
@variable(model, x_wind[1:W, 1:24] >= 0)  # Power output variable for wind generators
@variable(model, y[1:D, 1:24] >= 0)  # Demand variable
@variable(model, bat[1:B, 1:24])  # Battery variable
@variable(model, angle[1:N, 1:24])  # Voltage angles variable


# Add constraints for each plant
for g in 1:G
    for hour in 1:24
        @constraint(model, x_con[g, hour] <= con_generation[g, 6])  # Set the upper bound
        @constraint(model, x_con[g, hour] >= 0)  # Set the lower bound
    end
end

# Add constraints for each wind farm
for w in 1:W
    for hour in 1:24
        @constraint(model, x_wind[w, hour] <= wind_profile[hour, w+1])  # Set the upper bound
        @constraint(model, x_wind[w, hour] >= 0)  # Set the lower bound
    end
end

# Add constraints for each demand bid
for d in 1:D
    for hour in 1:24
        @constraint(model, y[d, hour] <= demand_bids_all[hour][d, 2])  # Set the upper bound
        @constraint(model, y[d, hour] >= 0)  # Set the lower bound
    end
end

#add constraints for initial con_generation
for g in 1:G
    @constraint(model, x_con[g, 1] <= con_generation[g, 11] + con_generation[g, 9])  # Set the upper bound
    @constraint(model, x_con[g, 1] >= con_generation[g, 11] - con_generation[g, 10])  # Set the lower bound
end

# Add temporary for each generator
for g in 1:G
    for hour in 2:24
        @constraint(model, x_con[g, hour] <= x_con[g, hour - 1] + con_generation[g, 7])  # Set the upper bound
        @constraint(model, x_con[g, hour] >= x_con[g, hour - 1] - con_generation[g, 8])  # Set the lower bound
    end
end

# add constraints for battery charging and discharging
for b in 1:B
    for hour in 1:24
        @constraint(model, bat[b, hour] <= battery[b, 5])  # Set the upper bound
        @constraint(model, bat[b, hour] >= -1 *battery[b, 5])  # Set the lower bound
    end
end

# add constraints for battery capacity
# sum all previous battery charging and discharging, they need to be below the battery capacity
for b in 1:B
    for hour in 2:24
        @constraint(model, sum(bat[b, h] for h in 1:hour) <= battery[b, 4])  # Set the upper bound
        @constraint(model, sum(bat[b, h] for h in 1:hour) >= 0)  # Set the lower bound
    end
end

# add constraints for transmission lines
for line in 1:L
    for hour in 1:24
        @constraint(model, transmission_lines[line, 3]*(angle[transmission_lines[line, 1],hour]-angle[transmission_lines[line, 2],hour]) <= transmission_lines[line, 4])  # Set the upper bound  #half the cap to set the different prices per zone
        @constraint(model, transmission_lines[line, 3]*(angle[transmission_lines[line, 1],hour]-angle[transmission_lines[line, 2],hour]) >= -1 * transmission_lines[line, 4])  # Set the lower bound
    end
end

balance = Vector{Any}(undef, N*24)

# add balance constraint for each node and each demand_hour_
for hour in 1:24
    for node in 1:N
        balance[(hour-1)*24+node] = @constraint(model, sum(x_con[g, hour] for g in 1:G if con_generation[g, 2] == node) +
                            sum(x_wind[w, hour] for w in 1:W if wind_generation[w, 2] == node)  +
                            sum(transmission_lines[line, 3]*(angle[transmission_lines[line, 1],hour]-angle[transmission_lines[line, 2],hour]) for line in 1:L if transmission_lines[line, 2] == node) 
                            ==
                            sum(y[d, hour] for d in 1:D if demand_bids_all[hour][d, 1] == node) +
                            sum(transmission_lines[line, 3]*(angle[transmission_lines[line, 1],hour]-angle[transmission_lines[line, 2],hour]) for line in 1:L if transmission_lines[line, 1] == node) +
                            sum(bat[b, hour] for b in 1:B if battery[b, 2] == node) )
    end
end

# Define the objective function
@objective(model, Max, sum(demand_bids_all[hour][d, 3] * y[d, hour] for d in 1:D, hour in 1:24) - sum(con_generation[g, 3] * x_con[g, hour] for g in 1:G, hour in 1:24) )


# Solve the model
optimize!(model)


# Check the status of the solution
status = termination_status(model)
if status == MOI.OPTIMAL
    println("Optimal solution found")

    # RETURN OBJECTIVE value
    println("Objective value: ", objective_value(model))


else
    println("No optimal solution found")
end


Optimal solution found
Objective value: 642912.292026483


In [8]:
value(angle[1,1])

559.1710133240356

In [9]:
# print decision variables
empty!(result_df)
for hour in 1:24
    resultvector = zeros(1+G+W+D+B+N)
    resultvector[1] = hour
    for g in 1:G
        resultvector[1+g] = value(x_con[g, hour])
    end
    for w in 1:W
        resultvector[1+G+w] = value(x_wind[w, hour])
    end
    for d in 1:D
        resultvector[1+G+W+d] = value(y[d, hour])
    end
    for b in 1:B
        resultvector[1+G+W+D+b] = value(bat[b, hour])
    end
    for node in 1:N
        resultvector[1+G+W+D+B+node] = value(angle[node,hour])
    end
    push!(result_df, resultvector)
end

In [10]:
#save the duals of the balance constraints to the equilibrium_df
empty!(equilibrium_df)
for hour in 1:24
    hour_results = []

    for node in 1:N
        push!(hour_results, dual(balance[(hour-1)*24+node]))
    end
    push!(equilibrium_df, [hour; hour_results...])
end




#save results to dataframe
CSV.write("results/optimization_results_marketprices_nodal_angles.csv", equilibrium_df)



"results/optimization_results_marketprices_nodal_angles.csv"

In [11]:
equilibrium_df

Row,hour,market_price_node1,market_price_node2,market_price_node3,market_price_node4,market_price_node5,market_price_node6,market_price_node7,market_price_node8,market_price_node9,market_price_node10,market_price_node11,market_price_node12,market_price_node13,market_price_node14,market_price_node15,market_price_node16,market_price_node17,market_price_node18,market_price_node19,market_price_node20,market_price_node21,market_price_node22,market_price_node23,market_price_node24
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1,12.2879,12.3696,12.261,12.3699,12.3415,12.3753,12.3817,12.3817,12.3703,12.3932,12.5123,12.3699,12.3924,13.1643,11.6468,10.9175,11.1986,11.2712,11.2525,11.6387,11.3499,11.258,12.3386,12.0237
2,2,12.2879,12.3696,12.261,12.3699,12.3415,12.3753,12.3817,12.3817,12.3703,12.3932,12.5123,12.3699,12.3924,13.1643,11.6468,10.9175,11.1986,11.2712,11.2525,11.6387,11.3499,11.258,12.3386,12.0237
3,3,10.3549,10.6211,10.2674,10.622,10.5295,10.6394,10.6606,10.6606,10.6232,10.698,11.0857,10.622,10.6952,13.2098,8.2664,5.89059,6.80634,7.0429,6.98192,8.2399,7.29916,7.0,10.52,9.49416
4,4,10.7075,11.0017,10.6108,11.0028,10.9005,11.022,11.0454,11.0454,11.004,11.0867,11.5152,11.0027,11.0836,13.8626,8.39952,5.77397,6.78599,7.0474,6.98002,8.37023,7.3306,7.0,10.89,9.75633
5,5,10.3549,10.6211,10.2674,10.622,10.5295,10.6394,10.6606,10.6606,10.6232,10.698,11.0857,10.622,10.6952,13.2098,8.2664,5.89059,6.80634,7.0429,6.98192,8.2399,7.29916,7.0,10.52,9.49416
6,6,8.85626,9.00354,8.80782,9.00406,8.95287,9.01369,9.02539,9.02539,9.00469,9.04609,9.26063,9.00405,9.04454,10.4359,7.7007,6.38617,6.89285,7.02373,6.98999,7.68603,7.16552,7.0,8.94761,8.38001
7,7,13.0103,13.32,12.9085,13.3211,13.2135,13.3413,13.3659,13.3659,13.3224,13.4095,13.8605,13.3211,13.4062,16.3314,10.5808,7.81706,8.88235,9.15752,9.08659,10.55,9.45563,9.10762,13.2024,12.009
8,8,13.0608,13.32,12.9924,13.3506,13.1889,13.3182,13.3503,13.3503,13.388,13.3126,13.335,13.3451,13.3396,13.2698,13.0978,13.2229,13.1747,13.1622,13.2503,13.2819,13.1487,13.1645,13.3391,13.0331
9,9,15.3638,15.6526,15.2688,15.6537,15.5533,15.6725,15.6955,15.6955,15.6549,15.7361,16.1568,15.6536,15.733,18.4614,13.0978,10.52,11.5136,11.7703,11.7041,13.069,12.0483,11.7237,15.543,14.4299


In [12]:
result_df

Row,hour,x_con1,x_con2,x_con3,x_con4,x_con5,x_con6,x_con7,x_con8,x_con9,x_con10,x_con11,x_con12,x_wind1,x_wind2,x_wind3,x_wind4,y1,y2,y3,y4,y5,y6,y7,y8,y9,y10,y11,y12,y13,y14,y15,y16,y17,x_bat1,x_bat2,x_bat3,x_bat4,x_bat5,x_angle1,x_angle2,x_angle3,x_angle4,x_angle5,x_angle6,x_angle7,x_angle8,x_angle9,x_angle10,x_angle11,x_angle12,x_angle13,x_angle14,x_angle15,x_angle16,x_angle17,x_angle18,x_angle19,x_angle20,x_angle21,x_angle22,x_angle23,x_angle24
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1,0.0,-4.1922e-13,0.0,0.0,0.0,93.0227,72.0,280.0,280.0,210.0,217.0,245.0,76.8921,101.54,92.8003,95.3709,67.4817,60.3784,165.153,88.6866,197.118,62.1542,207.773,113.653,79.9126,111.878,46.1717,44.3959,85.2401,78.1367,106.55,108.326,120.757,15.0,8.76885,10.0,-18.9077,5.0,559.171,-1309.15,1007.85,-907.105,489.407,-1413.61,-1815.41,-616.995,0.0,-185.825,991.604,1265.92,1419.58,4778.3,9146.48,8987.06,20412.9,24418.9,2325.21,244.729,23287.0,22734.8,3608.91,4152.72
2,2,0.0,0.0,0.0,0.0,0.0,57.0227,36.0,280.0,280.0,210.0,217.0,245.0,66.8277,90.9989,109.167,107.937,63.453,56.7737,155.293,113.547,185.349,58.4435,195.368,106.868,75.1417,105.198,43.4152,41.7454,80.1511,73.4719,100.189,101.859,113.547,15.0,1.23115,0.0,18.9077,-5.0,1281.58,-379.442,1679.49,0.0,1260.14,-471.782,-834.598,292.271,854.663,715.468,1772.71,2210.24,2376.62,4975.82,8853.39,9184.57,21092.4,25512.4,3022.06,1182.92,23889.3,23383.9,4558.56,4451.58
3,3,0.0,0.0,0.0,0.0,0.0,21.0227,0.0,280.0,280.0,200.332,198.904,197.0,78.422,116.959,142.88,134.78,60.4314,54.0702,147.898,108.14,176.523,55.6605,186.065,101.779,71.5635,100.189,41.3478,39.7575,76.3344,69.9732,95.418,97.0083,108.14,15.0,10.0,10.0,20.0,5.0,-7242.7,-8988.79,-6880.55,-8632.38,-7194.94,-9085.14,-9439.85,-8366.64,-7824.48,-7970.15,-6963.01,-6754.51,-6653.52,-3632.98,0.0,575.779,12557.4,17443.1,-5511.08,-7513.7,15619.7,14898.4,-4753.86,-4221.81
4,4,0.0,0.0,0.0,0.0,0.0,0.0,1.26477e-12,280.0,280.0,183.761,217.0,166.408,64.1437,134.249,159.57,133.664,59.4242,53.169,145.433,106.338,173.581,54.7328,182.964,100.083,70.3708,98.5191,40.6587,39.0949,75.0622,68.807,93.8277,95.3915,106.338,15.0,10.0,10.0,15.0,5.0,-24389.8,-26092.1,-24084.5,-25747.3,-24219.0,-26182.1,-26529.9,-25474.6,-24959.8,-25066.5,-24080.2,-23907.4,-23812.4,-20707.8,-17479.0,-16499.1,-4965.69,0.0,-22538.0,-24569.0,-1971.71,-2745.68,-21967.0,-21532.1
5,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,280.0,280.0,153.503,189.689,146.629,102.22,145.507,160.944,165.303,59.4242,53.169,145.433,106.338,173.581,54.7328,182.964,100.083,70.3708,98.5191,40.6587,39.0949,75.0622,68.807,93.8277,95.3915,106.338,15.0,10.0,10.0,20.0,5.0,-24225.3,-26099.2,-23890.3,-25758.7,-24100.7,-26198.4,-26561.4,-25506.1,-24976.6,-25112.6,-24179.5,-24137.5,-24145.2,-20807.1,-17425.2,-16598.4,-5129.44,0.0,-22736.1,-24880.9,-1794.32,-2947.2,-22485.3,-21392.1
6,6,0.0,0.0,0.0,0.0,0.0,0.5,0.0,280.0,280.0,166.169,145.0,194.629,134.039,131.112,157.023,161.828,60.4314,54.0702,147.898,108.14,176.523,55.6605,186.065,101.779,71.5635,100.189,41.3478,39.7575,76.3344,69.9732,95.418,97.0083,108.14,15.0,10.0,10.0,20.0,5.0,-3295.42,-5245.78,-2899.66,-4894.87,-3298.29,-5353.05,-5725.83,-4652.62,-4093.68,-4272.91,-3330.04,-3277.96,-3299.56,0.0,3480.01,4208.75,15857.7,20861.2,-1996.14,-4134.83,19165.3,18101.1,-1621.61,-434.471
7,7,10.4,41.348,0.0,0.0,0.0,36.5,31.8134,280.0,280.0,210.0,217.0,242.629,146.517,153.67,141.566,159.927,74.5321,66.6866,182.407,133.373,217.712,68.648,229.48,125.528,88.2616,123.566,50.9956,49.0343,94.1458,86.3003,117.682,119.644,133.373,-5.0,-10.0,0.0,1.7053e-13

In [13]:
#save result_df to csv
CSV.write("results/market_clearing_nodal_angles.csv", result_df)

"results/market_clearing_nodal_angles.csv"

In [14]:
#claculate capacities for each transmission line transmission lines as column and hours as rows
transmission_line_capacities = DataFrame(hour = Int[])

#add for each line a column
for line in 1:L
    transmission_line_capacities[!, Symbol("line_capacity_$line")] = Float64[]
end

for hour in 1:24
    hour_results = []

    for line in 1:L
        push!(hour_results, value(transmission_lines[line, 3]*(angle[transmission_lines[line, 1],hour]-angle[transmission_lines[line, 2],hour])))
    end
    push!(transmission_line_capacities, [hour; hour_results...])
end

#add the capcities from transmission_lines as a row below
cap = []
push!(cap, 99999)
for line in 1:L
    push!(cap, transmission_lines[line, 4])
end
push!(transmission_line_capacities, cap)


println(transmission_line_capacities)   

25×35 DataFrame
 Row │ hour   line_capacity_1  line_capacity_2  line_capacity_3  line_capacity_4  line_capacity_5  line_capacity_6  line_capacity_7  line_capacity_8  line_capacity_9  line_capacity_10  line_capacity_11  line_capacity_12  line_capacity_13  line_capacity_14  line_capacity_15  line_capacity_16  line_capacity_17  line_capacity_18  line_capacity_19  line_capacity_20  line_capacity_21  line_capacity_22  line_capacity_23  line_capacity_24  line_capacity_25  line_capacity_26  line_capacity_27  line_capacity_28  line_capacity_29  line_capacity_30  line_capacity_31  line_capacity_32  line_capacity_33  line_capacity_34 
     │ Int64  Float64          Float64          Float64          Float64          Float64          Float64          Float64          Float64          Float64          Float64           Float64           Float64           Float64           Float64           Float64           Float64           Float64           Float64           Float64           Float64           Fl